# Computer Assignment #1
## Ali Hamzehpour 810100129
## Implementation of Informed and Uninformed Search Algorithms and Solving a Problem with Them.
### Problem Description
We have a graph which each node contains either a student or a pizza station or nothing. Each student wants to have pizza from a unique pizza station and our task is to find the shortest path to deliver all pizzas to the students. we have some limitations: some edges in graph are shaky which means after we use them, we can't use them for some seconds. Also we only can carry one pizza at one time and some students' pizzas should be delivered sooner than some others. We have to implement BFS, IDS and A* algorithms and solve the problem with each of them.

### Graph Modeling
For modeling the problem's graph, First I implemented ```Edge``` and ```Node``` classes. ```Edge``` contains two nodes and a delay time if it's shaky. (otherwise it's set as ```None```) ```Node``` contains all of its edges and if the node has a pizza station or a student, it will have a delivery id as well (if student A wants to have pizza from station B, both of these nodes will have the same delivery id). Then I implemented a ```Graph``` class which contains all of the nodes and edges in our problem and it will hold order priorities as well.

In [2]:
from enum import Enum, auto
from dataclasses import dataclass

class EdgeType(Enum):
    NORMAL = auto()
    SHAKY = auto()
    
class NodeType(Enum):
    NORMAL = auto()
    PIZZA = auto()
    STUDENT = auto()    
    
@dataclass
class Edge:
    id: int
    type: EdgeType
    nodes: tuple[int, int]
    delay: int
    
    def find_dst(self, origin: int) -> int:
        if origin == self.nodes[0]:
            return self.nodes[1]
        return self.nodes[0]
    
@dataclass
class Node:
    id: int
    type: NodeType
    delivery_id: int
    edges: list[Edge]
    
def make_normal_node(id: int) -> Node:
    return Node(id, NodeType.NORMAL, None, [])

def make_normal_edge(id: int, n1: int, n2: int) -> Edge:
    return Edge(id, EdgeType.NORMAL, (n1, n2), None)
    
class Graph:
    def __init__(self, num_of_nodes: int):
        self.nodes: list[Node] = [make_normal_node(i) for i in range(1, num_of_nodes + 1)]
        self.edges: list[Edge] = []
        self.shaky_edges: list[Edge] = []
        self.num_of_deliveries = 0
        self.priorities: list[tuple(int, int)] = []
        
    def add_edge(self, n1: int , n2: int) -> None:
        new_edge: Edge = make_normal_edge(len(self.edges) + 1, n1, n2)
        self.edges.append(new_edge)
        self.nodes[n1 - 1].edges.append(new_edge)
        self.nodes[n2 - 1].edges.append(new_edge)
    
    def set_as_shaky(self, edge_id: int, delay: int) -> None:
        edge: Edge = self.edges[edge_id - 1]
        edge.type = EdgeType.SHAKY
        edge.delay = delay
        self.shaky_edges.append(edge)
    
    def set_delivery(self, std_node_id: int, pizza_node_id: int) -> None:
        self.num_of_deliveries += 1
        pizza_node: Node = self.nodes[pizza_node_id - 1]
        pizza_node.type = NodeType.PIZZA
        pizza_node.delivery_id = self.num_of_deliveries
        std_node: Node = self.nodes[std_node_id - 1]
        std_node.type = NodeType.STUDENT
        std_node.delivery_id = self.num_of_deliveries
        
    def add_priority(self, std1: int, std2: int) -> None:
        self.priorities.append((std1, std2))
                
    def get_node(self, node_id: int) -> Node:
        return self.nodes[node_id - 1]   
     

### Problem States
These attributes make two states different from each other:
* ```cur_node_id``` -> current node id which we are in the graph.
* ```cur_pizza``` -> current pizza we are carrying.
* ```handled_deliveries``` -> a set of delivery ids that are delivered.
* ```shaky_adj_edges``` -> the remaining time we should wait to use each edge that contains the current node which we are in.
<br>
 
States also have other attributes:
* ```shaky_edges_remain_time``` -> the remaining time of all the shaky edges
* ```cur_path``` -> the path to the current node
<br>

A goal state is a state that has delivered all the deliveries. The initial state is the state with the node id of the starting node and all other attributes are either 0, empty or None. ```find_next_state``` function will return all of the possible next states of the current state. I did some optimizations for these function: If we have stayed for some shaky edges we will use only those shaky edges for the next state and also we only stay on a node that has a shaky edge.


In [3]:
from typing import Any, Callable
from __future__ import annotations

class State:
    def __init__(self, cur_node_id: int , cur_pizza: int, handled_deliveries: set[int],
               shaky_edges_remain_time: dict[int, int], cur_path: list[int], graph: Graph):
        self.cur_node_id = cur_node_id
        self.cur_pizza = cur_pizza
        self.handled_deliveries = handled_deliveries
        self.shaky_edges_remain_time = shaky_edges_remain_time
        self.cur_path = cur_path
        self.shaky_adj_edges: dict[int, int] = self.find_shaky_adj_edges(graph)
    
    def __str__(self):
        return f"{self.cur_node_id} {self.cur_pizza} {self.handled_deliveries} {self.shaky_adj_edges}"
    
    def __eq__(self, other: Any) -> bool:
        return (isinstance(other, State) and
               self.cur_node_id == other.cur_node_id and
               self.cur_pizza == other.cur_pizza and
               self.handled_deliveries == other.handled_deliveries and
               self.shaky_adj_edges == other.shaky_adj_edges)
    
    def __hash__(self) -> int:
        return hash(str(self))

    def find_shaky_adj_edges(self, graph: Graph) -> tuple[int, int]:
        cur_node: Node = graph.get_node(self.cur_node_id)
        shaky_adj_edges = {}
        for edge in cur_node.edges:
            if edge.type == EdgeType.SHAKY:
                shaky_adj_edges[edge.id] = self.shaky_edges_remain_time[edge.id]
        return shaky_adj_edges

    def is_goal(self, total_deliveries: int) -> bool:
        return total_deliveries == len(self.handled_deliveries)

    def can_use_edge(self, edge: Edge) -> bool:
        return not(edge.type == EdgeType.SHAKY and self.shaky_adj_edges[edge.id] > 0)
    
    def is_stay_state(self) -> bool:
        return len(self.cur_path) > 1 and self.cur_path[-2] == self.cur_node_id
    
    def get_stay_state(self, updated_remain_time: dict[int, int], graph: Graph) -> State:
        return State(self.cur_node_id, self.cur_pizza, self.handled_deliveries.copy(), 
                    updated_remain_time.copy(), self.cur_path + [self.cur_node_id], graph)
        
    def calc_new_remain_time(self) -> dict[int, int]:
        updated_remain_time = self.shaky_edges_remain_time.copy()
        for shaky_edge in updated_remain_time:
            updated_remain_time[shaky_edge] = max(updated_remain_time[shaky_edge] - 1, 0) 
        return updated_remain_time
    
    def check_priority(self, delivery_id: int, graph: Graph) -> bool:
        priorities: list[tuple[int, int]] = graph.priorities
        for std1, std2 in priorities:
            if std2 == delivery_id and std1 not in self.handled_deliveries:
                return False
        return True     
    
    def find_states_of_new_position(self, dst: Node, edge: Edge, graph: Graph,
                                    updated_remain_time: dict[int, int]) -> list[State]:
        next_states: list[State] = []
        updated_remain_time_copy = updated_remain_time.copy()
        new_handled_deliveries = self.handled_deliveries.copy()
        new_pizza = self.cur_pizza
        if edge.type == EdgeType.SHAKY:
            updated_remain_time_copy[edge.id] = edge.delay
        if dst.type == NodeType.STUDENT:
            if dst.delivery_id == self.cur_pizza:
                new_handled_deliveries.add(self.cur_pizza)
                new_pizza = None
        elif dst.type == NodeType.PIZZA:
            if self.cur_pizza == None and self.check_priority(dst.delivery_id, graph):
                next_states.append(State(dst.id, dst.delivery_id, new_handled_deliveries, 
                    updated_remain_time.copy(), self.cur_path + [dst.id], graph))
        next_states.append(State(dst.id, new_pizza, new_handled_deliveries, 
                                    updated_remain_time_copy, self.cur_path + [dst.id], graph))
        return next_states

    def find_next_states(self, graph: Graph) -> list[State]:
        updated_remain_time: dict[int, int] = self.calc_new_remain_time()
        next_states: list[State] = []
        cur_node: Node = graph.get_node(self.cur_node_id)
        used_stay_state = False
        for edge in cur_node.edges:
            if self.is_stay_state() and edge.type != EdgeType.SHAKY:
                continue
            elif self.can_use_edge(edge):
                dst_id: int = edge.find_dst(self.cur_node_id)
                dst: Node = graph.get_node(dst_id)
                next_states.extend(self.find_states_of_new_position(dst, edge, graph, updated_remain_time.copy()))
            elif not used_stay_state:
                next_states.append(self.get_stay_state(updated_remain_time, graph))
        return next_states
    
def make_starting_state(starting_node_id: int, graph: Graph) -> State:
    shaky_edges_remain_time = {edge.id: 0 for edge in graph.shaky_edges} 
    return State(starting_node_id, None, set(),
                 shaky_edges_remain_time, [starting_node_id], graph)


### Breadth First Search
BFS first checks states that are closer to starting state. It'll always return the optimal path because between each state is one second difference. ```frontier``` is the queue of states. I add each state to ```explored``` set when I add them to ```frontier``` because ```explored``` is a set and it's faster to do all the search (to check if we've visited a state before or not) in it. The goal check is also done when we're adding a state to the ```frontier``` and not when a state comes on top of ```frontier```.

In [4]:
def BFS(starting_state: State, graph: Graph) -> tuple[State, int]:
    frontier: list[State] = [starting_state]
    explored: set[State] = set([starting_state])
    while frontier:
        cur_state = frontier.pop(0)
        next_states = cur_state.find_next_states(graph)
        for ns in next_states:
            if ns.is_goal(graph.num_of_deliveries):
                return ns, len(explored)
            if ns in explored: 
                continue
            frontier.append(ns)
            explored.add(ns)
    return None, len(explored)

### Iterative Deepening Search
IDS runs DFS in the states' graph but with depth limit. the depth limit starts from 0 and increases until we find the goal. Because of this method, IDS works like BFS and always finds the optimal path. The difference is in the memory. IDS only saves the states in our current path so it takes less memory than BFS. I implemented IDS in two ways. one recursively(```recursive_IDS```) and other with stack. (```stack_IDS```)

In [5]:
def recursive_IDS(starting_state: State, graph: Graph) -> (State, int):
    def IDS_helper(cur_state: State, explored: set[State], limit: int, num_of_explored: int) -> (State, int):
        num_of_explored += 1
        if limit == 0:
            return None, num_of_explored
        next_states: list[State] = cur_state.find_next_states(graph)
        new_explored: set[State] = explored.copy()
        new_states: list[State] = []
        for ns in next_states:
            if ns.is_goal(graph.num_of_deliveries):
                return ns, num_of_explored
            if ns in explored or limit == 1:
                continue
            new_explored.add(ns)
            new_states.append(ns)
        for ns in new_states:
            result, num_of_explored = IDS_helper(ns, new_explored, limit - 1, num_of_explored)
            if result:
                return result, num_of_explored
        return None, num_of_explored
    explored: set[State] = set()
    num_of_explored: int = 0
    depth_limit: int = 0
    MAX_LIMIT: int = 100
    while depth_limit < MAX_LIMIT:
        result, num_of_explored = IDS_helper(starting_state, explored, depth_limit, num_of_explored)
        if result:
            return result, num_of_explored
        depth_limit += 1
        explored = set()
    return None, num_of_explored

def stack_IDS(starting_state: State, graph: Graph) -> (State, int):
    depth_limit: int = 1
    MAX_DEPTH: int = 50
    num_of_explored: int = 0
    while depth_limit < MAX_DEPTH:
        frontier: list[tuple[State, int]] = [(starting_state, 0)]
        explored_sets: list[set[State]] = [set([starting_state])]
        while frontier:
            num_of_explored += 1
            cur_state, cur_depth = frontier.pop()
            cur_explored = explored_sets.pop()
            next_states = cur_state.find_next_states(graph)
            new_states = []
            for ns in next_states:
                if ns.is_goal(graph.num_of_deliveries):
                    return ns, num_of_explored
                if cur_depth == depth_limit - 1:
                    continue
                if ns in cur_explored:
                    continue
                cur_explored.add(ns)
                new_states.append(ns)
            for ns in new_states:
                frontier.append((ns, cur_depth + 1))
                explored_sets.append(cur_explored.copy())   
        depth_limit += 1
    return None, num_of_explored
        


### A*
A* is like BFS but we check states that has better estimated cost value. estimated cost value is the sum of current cost until the state and a heuristic function. heuristic function estimates how many seconds is remained until the goal state. A heuristic function is admissible if $$h(State1) \leq cost(State1toGoal)  $$and heuristic function is called consistent if $$ h(State1) - h(State2) \leq realcost(State1 to State2)$$
If a heuristic is admissible, the solution will be optimal in tree search and if a heuristic is consistent, the solution will be optimal in graph search.
the heuristic I use for this problem is:
* if we are carryig a pizza:
$$ h(state) = 2 \times (NumberOfAllDeliveries - NumberOfHandledDeliveries) - 1$$
* if not:
$$ h(state) = 2 \times (NumberOfAllDeliveries - NumberOfHandledDeliveries)$$
The logic behind it is that we need at least 2 steps for each delivery: one to reach the pizza station and the other to reach the student's node and if we already have a pizza, we'll decrease one step. So the heuristic is admissible. It's also consistent because for each step the heuristic increases at most one value(If we deliver a pizza in that step or get a pizza from station)but in reality each step takes one second so the heuristic between two states is always less than or equal to the real cost.

In [6]:
def calc_heuristic(state: State, graph: Graph) -> int:
    has_pizza: int = 0
    if state.cur_pizza:
        has_pizza = 1
    return 2 * (graph.num_of_deliveries - len(state.handled_deliveries)) - has_pizza

I created a wrapper class for python's ```heapq``` library because it didn't have the option to use a custom compare function and also it wasn't object-oriented. ```MinHeap``` class takes some initial values for heap and a key function for comparison.

In [7]:
import heapq

class MinHeap:
    def __init__(self, initial: list[Any] = None, key: Callable[[Any], int] = lambda x :x):
        self.key = key
        if initial:
            self._data = [[key(item), i, item] for i, item in enumerate(initial)]
            self.index = len(self._data)
            heapq.heapify(self._data)
        else:
            self._data = []

    def push(self, item):
        heapq.heappush(self._data, [self.key(item), self.index, item])
        self.index += 1

    def pop(self):
        return heapq.heappop(self._data)[-1]
    
    def __iter__(self):
        for i in self._data:
            yield i[-1]

The implementation of A* is the same as BFS but ```frontier``` is a min-heap which every time we pop the state with lowest estimated cost. The function also has ```alpha``` parameter which is for weighted A*. Weighted A* is the same as normal A* but we multiply heuristic value with the constant value $\alpha$. It makes the search faster but it could lose its optimality.

In [8]:
def A_star(starting_state: State, graph: Graph, alpha: int = 1) -> tuple[State, int]:
    
    def estimate_cost(state: State) -> int:
        return alpha * calc_heuristic(state, graph) + len(state.cur_path)
    
    frontier: MinHeap = MinHeap([starting_state], estimate_cost)
    explored: set[State] = set([starting_state])
    while frontier:
        cur_state = frontier.pop()
        next_states = cur_state.find_next_states(graph)
        for ns in next_states:
            if ns.is_goal(graph.num_of_deliveries):
                return ns, len(explored)
            if ns in explored:
                continue
            frontier.push(ns)
            explored.add(ns)
    return None, len(explored)
    

```read_input``` is a function that simply reads input from a file and saves it in a graph class and returns it with the id of the node in which we should start our search.

In [9]:
def read_input(input_file_name: str) -> (Graph, int):
    input_file = open(input_file_name, "r")
    n, m = map(int, input_file.readline().split())
    graph = Graph(n)
    for i in range(m):
        n1, n2 = map(int, input_file.readline().split())
        graph.add_edge(n1, n2)
    h = int(input_file.readline())
    for i in range(h):
        edge_id, delay = map(int, input_file.readline().split())
        graph.set_as_shaky(edge_id, delay)
    starting_node_id = int(input_file.readline())
    s = int(input_file.readline())
    for i in range(s):
        std_node, pizza_node = map(int, input_file.readline().split())
        graph.set_delivery(std_node, pizza_node)
    t = int(input_file.readline())
    for i in range(t):
        s1, s2 = map(int, input_file.readline().split())
        graph.add_priority(s1, s2)
    return (graph, starting_node_id)

```test_search_func``` is a wrapper function which takes a search function and does the search in the graph using it.

In [10]:
import time
def test_search_func(search_func: Callable[[State, Graph], Tuple[State, int]],
                     starting_state: State, graph: Graph) -> None:
    REPEAT: int = 3
    time_sum: int = 0
    for i in range(REPEAT):
        start: int = time.time()
        goal_state, explored_nodes = search_func(starting_state, graph)
        end: int = time.time()
        time_sum += (end - start)
    print("Average time spent: ", time_sum / REPEAT)
    print("Found path:", end = " ")
    print(*goal_state.cur_path, sep="->")
    print("Length of found path: ", len(goal_state.cur_path))        
    print("Number of explored nodes: ", explored_nodes)

### Testing
for each test I did the search with all of the algorithms: (Because IDS algorithm took too long I tested it in separate cells.)

In [11]:
from functools import partial
algorithms ={"BFS": BFS,
              "A*": partial(A_star, alpha = 1),
              "A* with alpha = 2": partial(A_star, alpha = 2),
              "A* with alpha = 4": partial(A_star, alpha = 4),
          #    "IDS": IDS
            }
def run_test(test_number: int)-> None:
    print(f"TEST{test_number}: ")
    input_file_name = f"./tests/Test{test_number}.txt"
    graph, starting_node_id = read_input(input_file_name)
    starting_state = make_starting_state(starting_node_id, graph)
    for alg in algorithms:
        print(f"{alg}: ")
        alg_func = algorithms[alg]
        test_search_func(alg_func, starting_state, graph)
        print("######")


### Test 1

In [12]:
run_test(1)

TEST1: 
BFS: 
Average time spent:  0.012333552042643229
Found path: 6->4->10->5->3->8->9->1->7->9->2
Length of found path:  11
Number of explored nodes:  613
######
A*: 
Average time spent:  0.005999962488810222
Found path: 6->4->10->9->1->3->8->9->7->9->2
Length of found path:  11
Number of explored nodes:  390
######
A* with alpha = 2: 
Average time spent:  0.0010066032409667969
Found path: 6->4->10->9->1->3->8->9->7->9->2
Length of found path:  11
Number of explored nodes:  102
######
A* with alpha = 4: 
Average time spent:  0.001011212666829427
Found path: 6->4->10->9->1->3->8->9->7->9->2
Length of found path:  11
Number of explored nodes:  103
######


For IDS:

In [13]:
print("IDS recursive TEST1:")
INPUT_FILE = "./tests/Test1.txt"
graph, starting_node_id = read_input(INPUT_FILE)
starting_state = make_starting_state(starting_node_id, graph)
test_search_func(recursive_IDS, starting_state, graph)
print("IDS non-recursive TEST1:")
test_search_func(stack_IDS, starting_state, graph)

IDS recursive TEST1:
Average time spent:  0.25533318519592285
Found path: 6->4->10->5->3->8->9->1->7->9->2
Length of found path:  11
Number of explored nodes:  12704
IDS non-recursive TEST1:
Average time spent:  0.2690231005350749
Found path: 6->4->10->9->7->9->2->9->1->3->8
Length of found path:  11
Number of explored nodes:  12742


| Alg | found path length | explored nodes | Run Time |
| :-: | :----------------:| :------------: | :------: |
| BFS | 11    | 613                        | 0.010689179102579752  |
| IDS recursive |    11   |            12704                |     0.24799895286560059     |
| IDS non-recursive |    11   |        12742                    |     0.2346787452697754     |
| A*  | 11    | 390                         | 0.008892695109049479   |
| A* Alpha=2  | 11         | 102                   | 0.0009995301564534504   |
| A* Alpha=4  | 11         | 103                  | 0.0010000069936116536   |

### Test 2

In [14]:
 run_test(2)

TEST2: 
BFS: 
Average time spent:  0.10967294375101726
Found path: 14->9->4->15->12->3->12->6->14->1->14->12->3->8->6->4->10->11->8->3->13
Length of found path:  21
Number of explored nodes:  3599
######
A*: 
Average time spent:  0.0690005620320638
Found path: 14->9->4->15->12->3->12->6->14->1->14->12->3->8->6->4->10->11->8->3->13
Length of found path:  21
Number of explored nodes:  3213
######
A* with alpha = 2: 
Average time spent:  0.038999716440836586
Found path: 14->9->4->15->12->3->12->6->8->6->4->9->14->1->14->12->3->8->11->8->3->13
Length of found path:  22
Number of explored nodes:  1794
######
A* with alpha = 4: 
Average time spent:  0.009000142415364584
Found path: 14->9->4->15->12->3->12->6->8->6->4->9->14->1->14->12->3->8->11->8->3->13
Length of found path:  22
Number of explored nodes:  558
######


For IDS it took too long to respond, in fact it checks about 90 million states when it reaches 20th depth! The reason is because we only keep our current path in explored set, we may see the same states in different path in our states' graph and it makes IDS to check too many states.

In [15]:
# print("IDS recursive TEST2:")
# INPUT_FILE = "./tests/Test2.txt"
# graph, starting_node_id = read_input(INPUT_FILE)
# starting_state = make_starting_state(starting_node_id, graph)
# test_search_func(recursive_IDS, starting_state, graph)
# print("IDS non-recursive TEST2:")
# test_search_func(stack_IDS, starting_state, graph)

| Alg | found path length | explored nodes | Run Time |
| :-: | :----------------:| :------------: | :------: |
| BFS | 21    | 3599                        | 0.0766916275024414  |
| IDS |       |                            |          |
| A*  | 21    | 3213                         | 0.0667714277903239   |
| A* Alpha=2  | 22         | 1794                   | 0.04497575759887695   |
| A* Alpha=4  | 22         | 558                  | 0.009969711303710938   |

### Test 3

In [16]:
run_test(3)

TEST3: 
BFS: 
Average time spent:  1.9303345680236816
Found path: 15->14->8->20->13->8->18->10->8->9->16->5->15->12->3->19->11->19->13->6->13->1->13->8->17->8->18->2->18->4->14->16
Length of found path:  32
Number of explored nodes:  53972
######
A*: 
Average time spent:  1.7310007413228352
Found path: 15->14->8->20->13->8->18->10->8->9->16->5->15->12->3->19->11->19->13->6->13->1->13->8->17->8->18->2->18->4->14->16
Length of found path:  32
Number of explored nodes:  50331
######
A* with alpha = 2: 
Average time spent:  0.1600064436594645
Found path: 15->14->8->20->13->8->18->10->8->9->16->5->15->12->3->19->11->19->13->6->13->1->13->8->17->8->18->2->18->4->14->16
Length of found path:  32
Number of explored nodes:  7417
######
A* with alpha = 4: 
Average time spent:  0.015660206476847332
Found path: 15->14->8->20->8->18->2->18->10->8->9->16->5->15->12->3->19->13->8->18->4->14->16->14->4->1->13->8->17->19->11->19->13->6
Length of found path:  34
Number of explored nodes:  858
######


For this test IDS was unable to find the answer in a normal time:

In [17]:
# print("IDS recursive TEST3:")
# INPUT_FILE = "./tests/Test3.txt"
# graph, starting_node_id = read_input(INPUT_FILE)
# starting_state = make_starting_state(starting_node_id, graph)
# test_search_func(recursive_IDS, starting_state, graph)
# print("IDS non-recursive TEST2:")
# test_search_func(stack_IDS, starting_state, graph)

| Alg | found path length | explored nodes | Run Time |
| :-: | :----------------:| :------------: | :------: |
| BFS | 32    | 53972                        | 1.79891037940979  |
| IDS |       |                            |          |
| A*  | 32    | 50331                         | 1.571475585301717   |
| A* Alpha=2  | 32         | 7417                   | 0.17083978652954102   |
| A* Alpha=4  | 34         | 858                  | 0.01749420166015625   |

### Testing with additional tests

In [18]:
run_test(0)

TEST0: 
BFS: 
Average time spent:  0.0003333886464436849
Found path: 1->2->3->4->4->4->3->2->3->5
Length of found path:  10
Number of explored nodes:  30
######
A*: 
Average time spent:  0.00033362706502278644
Found path: 1->2->3->4->4->4->3->2->3->5
Length of found path:  10
Number of explored nodes:  29
######
A* with alpha = 2: 
Average time spent:  0.00033322970072428387
Found path: 1->2->3->4->4->4->3->2->3->5
Length of found path:  10
Number of explored nodes:  24
######
A* with alpha = 4: 
Average time spent:  0.0003333091735839844
Found path: 1->2->3->4->4->4->3->2->3->5
Length of found path:  10
Number of explored nodes:  21
######


In [19]:
run_test(4)

TEST4: 
BFS: 
Average time spent:  0.016669511795043945
Found path: 5->7->2->9->4->8->10->3->8->1->6->2
Length of found path:  12
Number of explored nodes:  731
######
A*: 
Average time spent:  0.009999752044677734
Found path: 5->7->2->9->4->8->10->3->8->1->6->2
Length of found path:  12
Number of explored nodes:  548
######
A* with alpha = 2: 
Average time spent:  0.0016668637593587239
Found path: 5->3->8->4->8->10->1->7->2->9->1->6->2
Length of found path:  13
Number of explored nodes:  145
######
A* with alpha = 4: 
Average time spent:  0.0016693274180094402
Found path: 5->3->8->4->8->10->1->7->2->9->1->6->2
Length of found path:  13
Number of explored nodes:  135
######


In [20]:
run_test(5)

TEST5: 
BFS: 
Average time spent:  0.10499803225199382
Found path: 2->14->11->9->15->6->9->14->1->5->8->10->6->4->6->9->7
Length of found path:  17
Number of explored nodes:  4690
######
A*: 
Average time spent:  0.046333630879720054
Found path: 2->14->11->9->15->6->9->14->1->5->8->10->6->4->6->9->7
Length of found path:  17
Number of explored nodes:  2605
######
A* with alpha = 2: 
Average time spent:  0.0066721439361572266
Found path: 2->4->6->9->11->9->15->6->4->1->5->8->10->6->4->6->9->7
Length of found path:  18
Number of explored nodes:  565
######
A* with alpha = 4: 
Average time spent:  0.00966032346089681
Found path: 2->14->11->9->15->6->9->14->1->5->8->10->6->4->6->9->7
Length of found path:  17
Number of explored nodes:  350
######


In [21]:
run_test(6)

TEST6: 
BFS: 
Average time spent:  0.03266684214274088
Found path: 11->6->5->10->7->10->5->1->2->9->14->6->8->4->2->1->3->7->13
Length of found path:  19
Number of explored nodes:  1843
######
A*: 
Average time spent:  0.029666344324747723
Found path: 11->6->5->10->7->10->5->1->2->9->14->6->8->3->1->1->3->7->13
Length of found path:  19
Number of explored nodes:  1465
######
A* with alpha = 2: 
Average time spent:  0.005001942316691081
Found path: 11->6->5->10->7->10->5->1->2->9->14->6->8->3->1->1->3->7->13
Length of found path:  19
Number of explored nodes:  315
######
A* with alpha = 4: 
Average time spent:  0.004997730255126953
Found path: 11->6->5->10->7->9->14->6->5->10->5->1->2->1->3->7->13->7->3->8->3->1
Length of found path:  22
Number of explored nodes:  304
######


### Comparison and Conclusion
BFS, IDS and A*(with consistent heuristic) always find the optimal path for us but weighted A* may not. But the advantage of weighted A* is its speed and memory. Because it visits less states, it needs less time and space to find a path that is at least near to an optimal path for us. With playing with $\alpha$ constant we can find a proper value for it that has a good trade-off between optimality and speed. (in this problem $\alpha = 2$ seems to work better) IDS is much slower than A* and BFS but its advantage is that it uses much less memory than other algorithms. IDS was not a good choice for this problem because we had too much state and since IDS save only its current path, it may visit some states more than once so it becomes too slow. In general for problems that have smaller space, we can use IDS. The disadvantage of A* in my opinion is that we have to find a good heuristic in order to make A* work well and it's not always an easy task. If our heuristic is not good enough, the algorithm's speed may become slower than BFS because it needs min-heap for its frontier queue.

### Sources I Used for The Project:

* [min-heap implementation](https://stackoverflow.com/questions/8875706/heapq-with-custom-compare-predicate)
* [IDS algorithm](https://www.geeksforgeeks.org/iterative-deepening-searchids-iterative-deepening-depth-first-searchiddfs/)
* [python typing](https://docs.python.org/3/library/typing.html)